In [64]:
import os, sys
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import metric_learn
import matplotlib.pyplot as plt
import itertools
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


sys.path.insert(0, os.path.dirname(os.getcwd())+ '\\source')
from structure_analysis import performance_analysis



In [90]:
MATCHSCORE_DIR = os.path.dirname(os.path.dirname(os.getcwd())) + "\\alignments\\match_4\\"

file_structure = {
    "name": "Mozart",
    "structure": {
        "a1_1": (1, 5),
        "a2_1": (5, 9),
        "a1_2": (9, 13),
        "a2_2": (13, 17),
        "a1_3": (17, 21),
        "a2_3": (21, 27),
        "a1_4": (27, 31),
        "a2_4": (31, 36)
    },
    "step": 1,
    "measure": 6
}

def performance_array_to_attr(Pn, t=10):
    f = lambda i : np.hstack((Pn[:, 0 + i*t : t + i*t], Pn[:, 4*t + i*t: 5*t+i*t], Pn[:,8*t+i*t: 9*t +i*t], Pn[:,12*t+i*t: 13*t+i*t]))
    P_tempo = f(0)
    P_vel = f(1)
    P_timing = f(2)
    P_artic = f(3)
    return P_tempo, P_vel, P_timing, P_artic

def uniformize_windows(X, samples_per_window):
    # the last is the number of windows
    X_new = np.zeros([len(X),len(max(X, key = lambda x: len(x))), samples_per_window*4]) 
    for i, j in enumerate(X):
        X_new[i][0:len(j)] = j
    X = X_new.reshape(X_new.shape[0], X_new.shape[1]*X_new.shape[2])
    return X

def combinations_of_attributes(P1, P2, labels, file_structure, samples_per_window):
    data = zip(P1, P2)
    X1 = list()
    X2 = list()
    y = list()
    z = list()
    k = 0
    for p1, p2 in data:
        for section_name, section_coord in file_structure["structure"].items():
            step = file_structure["step"]
            temp1 = p1[int((section_coord[0]-1)*6/step) : int((section_coord[1]-1)*6/step)].tolist()
            temp2 = p2[int((section_coord[0]-1)*6/step) : int((section_coord[1]-1)*6/step)].tolist()
            X1.append(temp1)
            X2.append(temp2)
            y.append(labels[k])
            z.append(section_name)
        k +=1
    
    X1 = uniformize_windows(X1, samples_per_window)
    X2 = uniformize_windows(X2, samples_per_window)
    
    return X1, X2, y, z


def attribute_combinations_to_html(data, labels, file_structure, samples_per_window):
    P_tempo, P_vel, P_timing, P_artic = zip(*[performance_array_to_attr(perf, samples_per_window) for perf in data])
    performance_dict = {
        "tempo" : P_tempo,
        "velocity" : P_vel,
        "timing" : P_timing,
        "articulation" : P_artic
    }
    for combi in itertools.combinations(performance_dict, 2):
        P1 = performance_dict[combi[0]]
        P2 = performance_dict[combi[1]]
        X1, X2, y, z = combinations_of_attributes(P1, P2, labels, file_structure, samples_per_window)
        cov = metric_learn.Covariance()
        X_cov1 = cov.fit_transform(X1)
        X_cov2 = cov.fit_transform(X2)

        pca = PCA(whiten=True)
        X_pca1 = pca.fit_transform(X_cov1)
        X_pca2 = pca.fit_transform(X_cov2)
        X = TSNE(n_components=1).fit_transform(X_pca1).reshape(X_pca1.shape[0])
        Y = TSNE(n_components=1).fit_transform(X_pca2).reshape(X_pca2.shape[0])
        fig = go.Figure(px.scatter_3d(x=X, y=Y, z=z, color=y))
        
        title_text = 'Covariance_Sampling ' + combi[0] + ' vs ' + combi[1]
        fig.update_layout(title_text=title_text)
        pio.write_html(fig, file='html_files\\'+title_text+'.html', auto_open=True)

In [86]:
file = "Mozart_K331_1st-mov_p08.match"
window_size = 6 
samples_per_window = 10
data = list()
labels = list()


for file in os.listdir(MATCHSCORE_DIR):
    if file.startswith("Mozart") and file.endswith(".match"):
        P = performance_analysis(MATCHSCORE_DIR + file, file_structure["step"], window_size, samples_per_window)
        data.append(P.reshape(P.shape[0], P.shape[1]*P.shape[2]*P.shape[3]))
        labels.append(os.path.splitext(file)[0][-3:])
        


In [ ]:
X = list()
y = list()
z = list()
k = 0
for perf in data:
    for section_name, section_coord in file_structure["structure"].items():
        step = file_structure["step"]
        temp = perf[int((section_coord[0]-1)*6/step) : int((section_coord[1]-1)*6/step)].tolist()
        X.append(temp)
        y.append(labels[k])
        z.append(section_name)
    k +=1

# Uniformize all windows
# the last is the number of attribute vector * the number of windows

X_new = np.zeros([len(X),len(max(X, key = lambda x: len(x))), samples_per_window*4*4]) 
for i, j in enumerate(X):
    X_new[i][0:len(j)] = j
    
X = X_new.reshape(X_new.shape[0], X_new.shape[1]*X_new.shape[2])

In [ ]:
cov = metric_learn.Covariance()
X_cov = cov.fit_transform(X)

pca = PCA(whiten=True)
X_pca = pca.fit_transform(X_cov)

tsne = TSNE()
X_embedded = tsne.fit_transform(X_pca)
fig = px.scatter_3d(x=X_embedded[:, 0], y=X_embedded[:, 1], z=z, color=y)
fig.show()